In [ ]:
from data_manager.data_manager import DataManager
from analysis.analysis import Analysis

# Load the data
file_path = 'self_report_data.xlsx'  # Update with your file path
manager = DataManager(file_path)
data = manager.load_data()

# Initialize the Analysis class
analysis = Analysis(data)

# Define metrics
metrics = ['SUS', 'S-UEQ', 'NASA-TLX']

# Perform analysis
ttest_results = analysis.analyze_metrics(metrics)

# Summarize results
analysis.summarize_results(ttest_results)

In [2]:
print("alo")
a=13

alo
